In [1]:
import os

list_file = sorted(list(os.walk('./file/1.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[1]

file

'crawler_2.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/1.part_number/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：718


['44136',
 '86043',
 '90001',
 '90002',
 '90003',
 '90004',
 '90005',
 '90007',
 '90009',
 '90010',
 '90011',
 '90012',
 '90014',
 '90015',
 '90016',
 '90017',
 '90030',
 '90031',
 '90033',
 '90035',
 '90036',
 '90037',
 '90039',
 '90040',
 '90041',
 '90042',
 '90043',
 '90044',
 '90045',
 '90046',
 '90047',
 '90049',
 '90050',
 '90051',
 '90052',
 '90053',
 '90054',
 '90055',
 '90056',
 '90057',
 '90058',
 '90060',
 '90061',
 '90062',
 '90065',
 '90066',
 '90068',
 '90069',
 '90070',
 '90071',
 '90072',
 '90073',
 '90074',
 '90075',
 '90076',
 '90077',
 '90078',
 '90079',
 '90080',
 '90081',
 '90082',
 '90468',
 '90479',
 '90483',
 '90484',
 '90485',
 '90489',
 '90490',
 '90495',
 '90496',
 '90498',
 '90500',
 '90503',
 '90504',
 '90508',
 '90551',
 '90569',
 '90582',
 '90610',
 '90611',
 '90622',
 '90624',
 '90629',
 '90692',
 '90701',
 '90719',
 '90729',
 '90735',
 '90737',
 '90740',
 '90741',
 '90743',
 '90744',
 '90745',
 '90746',
 '90749',
 '90752',
 '90756',
 '90760',
 '90761',


In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = f'''https://ecatalog.smpcorp.com/v2/fs/api/part/partselect?part={input_.loc[a, 'Part Number']}&func=PART&vid='''
        check_url = input_.loc[a, 'Url']
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            dict_dict = resp.json()
            if dict_dict:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
dict_attr, dict_ = {}, dict_dict['pd']
for key, value in dict_.items():
    if key not in ['rowId', 'basePart', 'maxRows'] and value != None:
        if type(value) in [int, float]:
            dict_attr[key.strip().title()] = value
        elif type(value) == str:
            dict_attr[key.strip().title()] = value.strip()
        else:
            print('fuck')

dict_attr

{'Site': 'FS',
 'Webbase': '44136',
 'Brand': 'FS',
 'Partdesc_En': 'Evaporator/Heater Core',
 'Partlongdesc_En': 'Tube & Fin Evaporator Core/Heater Core Combination',
 'Pop': 'F',
 'Qtyeach': 1}

In [9]:
vehicle = '\n'.join([dict_vehicle['buyersGuideDesc'].strip() for dict_vehicle in dict_dict['pp']['buyersGuides']])

print(vehicle)

Freightliner Century Class (03-00)
Freightliner Columbia (03-00)


In [10]:
list_spec, dict_count = dict_dict['pp']['partSpecs'], {}
for i, spec in enumerate(list_spec):
    key = spec['attributeName_en'].strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_spec, dict_no = {}, {}
for i, spec in enumerate(list_spec):
    key = spec['attributeName_en'].strip()
    if dict_count[key] == 1:
        dict_spec[str(i)] = {key: spec['siteValue_en'].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_spec[str(i)] = {f'''{key}-{dict_no[key]}''': spec['siteValue_en'].strip()}

dict_spec

{'0': {'Outlet Gender': 'Male'},
 '1': {'Description': 'Evaporator Cores'},
 '2': {'Warranty': '120 days'},
 '3': {'Core Depth Fraction': '4-1/2'},
 '4': {'Core Height Fraction': '9'},
 '5': {'Inlet Gender': 'Male'},
 '6': {'Core Width Fraction': '5-1/2'},
 '7': {'Core Depth MM': '114.3'},
 '8': {'Core Depth IN': '4.500'},
 '9': {'Core Height MM': '228.6'},
 '10': {'Core Height IN': '9.000'},
 '11': {'Core Width IN': '5.500'},
 '12': {'Core Width MM': '139.7'},
 '13': {'Type': 'Tube & Fin'}}

In [11]:
service = '\n'.join([f'''{i+1}. {service['serviceName'].strip()}''' for i, service in enumerate(dict_dict['pp']['completeRepair']['serviceSpec'])])

print(service)

1. Fuel Filter Flush (Required)
2. Oil Change (Required)


In [12]:
import json

df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
df_temp.loc[0, 'Vehicle'] = vehicle
df_temp.loc[0, 'Service'] = service
df_temp = pd.concat([df_temp, pd.DataFrame([dict_attr])], axis=1).fillna('')
df_temp.loc[0, 'Json_Spec'] = json.dumps(dict_spec)

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Part Number,Url,Vehicle,Service,Site,Webbase,Brand,Partdesc_En,Partlongdesc_En,Pop,Qtyeach,Json_Spec
0,1,44136,https://www.4s.com/en/ecatalog?part=44136&type...,Freightliner Century Class (03-00)\nFreightlin...,1. Fuel Filter Flush (Required)\n2. Oil Change...,FS,44136,FS,Evaporator/Heater Core,Tube & Fin Evaporator Core/Heater Core Combina...,F,1,"{""0"": {""Outlet Gender"": ""Male""}, ""1"": {""Descri..."


In [13]:
crawler_status = 'ok'

crawler_status

'ok'

In [14]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    df_temp.loc[0, 'Check_Url'] = check_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [15]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,Part Number,Url,Vehicle,Service,Site,Webbase,Brand,Partdesc_En,Partlongdesc_En,Pop,Qtyeach,Json_Spec
0,1,44136,https://www.4s.com/en/ecatalog?part=44136&type...,Freightliner Century Class (03-00)\nFreightlin...,1. Fuel Filter Flush (Required)\n2. Oil Change...,FS,44136,FS,Evaporator/Heater Core,Tube & Fin Evaporator Core/Heater Core Combina...,F,1,"{""0"": {""Outlet Gender"": ""Male""}, ""1"": {""Descri..."


In [16]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
